<img src="https://i.imgur.com/3VyrCXI.png" width="1200">

## 도시, 숲, 데이터
* 시민이 참여해서 데이터가 만들어지면 지역사회에 어떤 변화를 만들 수 있을까?


## 라이브러리 로드

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium

## 폰트 설정

In [ ]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    
    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

plt.rc("font", family=get_font_family())
plt.rc("axes", unicode_minus=False)

In [ ]:
pd.options.display.max_columns = None

## 데이터 로드

In [ ]:
# read_excel 을 통해 데이터를 읽어옵니다.
df = pd.read_csv("https://raw.githubusercontent.com/corazzon/chuncheon_tree/main/cc_tree.csv")
df.shape

In [ ]:
# 데이터의 일부를 살펴봅니다.
df.head()

## 데이터 요약

In [ ]:
# 행과 열의 수, 컬럼, 결측치, 데이터 타입, 메모리 사용량 등을 볼 수 있습니다.
df.info()

## 결측치 확인

In [ ]:
# 결측치
df.isnull().sum()

In [ ]:
# 결측치 시각화
plt.figure(figsize=(15, 6))
sns.heatmap(df.isnull(), cmap="Greys_r")

## 기술통계
### 수치 데이터

In [ ]:
# describe 는 기본값으로 수치데이터가 있다면 수치데이터에 대한 기술통계 값을 보여줍니다.
df.describe()

In [ ]:
_ = df.hist(figsize=(12, 10), bins=50)

### 범주형 데이터

In [ ]:
# 반도수, 유일값 빈도수, 최빈값, 최빈값에 대한 빈도수
df.describe(include="object")

In [ ]:
# 유일값 빈도수
df.nunique()

## 수목 수령

In [ ]:
df["수목 수령"].describe()

In [ ]:
# 수목 수령이 500 이상인 데이터를 찾습니다.
df[df["수목 수령"] > 500]

In [ ]:
# 수목 수령에 대한 빈도수를 구하고 시각화 합니다.
df["수목 수령"].value_counts().sort_index().plot.bar(figsize=(15, 4), rot=0)

In [ ]:
# 수목 수령에 대한 히스토그램을 그립니다.
df["수목 수령"].hist(bins=100)

## 수종명 빈도

In [ ]:
# 수종별 빈도수
tree_count = df["수종명(한글명)"].value_counts()
tree_count.to_frame(name="count").T

In [ ]:
plt.figure(figsize=(15, 8))
sns.countplot(data=df, y="수종명(한글명)", order=tree_count.index, palette="Greens_r")
plt.title("수종명 빈도수")

In [ ]:
df["수종관리번호"].value_counts()

In [ ]:
df[df["수종관리번호"].str.contains("미식재")]

## 경도, 위도별 수종

In [ ]:
# plt.colormaps()

In [ ]:
# 경도, 위도별 수종명
plt.figure(figsize=(15, 8))
sns.scatterplot(data=df, x="경도", y="위도", hue="수종명(한글명)", palette="tab20c_r")
plt.legend(bbox_to_anchor=(1, 1))

## 법정동별 수종 빈도

In [ ]:
# 법정동별 수종 빈도수
pd.crosstab(df["법정동 코드"], df["수종명(한글명)"]).style.format("{:,}")

In [ ]:
# pd.crosstab을 통해 두 개의 변수에 대한 빈도수 구하기
load_tree = pd.crosstab(df["도로명코드"], df["수종명(한글명)"])
load_tree.shape

## 도로명코드와 수종 빈도

In [ ]:
# "도로명코드"와 "수종명(한글명)" 에 대한 heatmap 표현하기 
plt.figure(figsize=(20, 8))
sns.heatmap(load_tree.T, cmap="Greys")

In [ ]:
# 데이터 프레임 형태로 "도로명코드"와 "수종명(한글명)" 보기
load_tree.style.format("{:,}")

## 조성 일자

In [ ]:
# 조성 일자는 대부분 결측치
df["조성 일자"].value_counts()

## 수고

In [ ]:
tree_high_mean = df.groupby("수종명(한글명)")["수고"].mean().sort_values(ascending=False)
tree_high_mean

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(data=df, y="수종명(한글명)", x="수고", ci=None, order=tree_high_mean.index, palette="Greens_r")
plt.title("평균 수고")

## 수고, 수관폭, 흉고직경

In [ ]:
plt.figure(figsize=(15, 10))
sns.scatterplot(data=df, x="수고", y="수관폭", hue="수종명(한글명)")
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
plt.figure(figsize=(15, 10))
sns.scatterplot(data=df, x="수고", y="흉고직경", hue="수종명(한글명)")
plt.legend(bbox_to_anchor=(1,1))

## 조성위치

In [ ]:
plt.figure(figsize=(15, 10))
sns.scatterplot(data=df, x="수고", y="수관폭", hue="조성위치\n(좌/우/중앙분리대)")
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(data=df, y="수종명(한글명)", hue="조성위치\n(좌/우/중앙분리대)")

In [ ]:
df.columns

## 지도 시각화

In [ ]:
lat_long = df[["위도", "경도"]].mean()
lat_long.values

In [ ]:
# folium 에 색상을 표현하기 위해 컬러 목록 가져오기
import matplotlib.colors as mcolors

colors = mcolors.CSS4_COLORS 
by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgb(color))),
                 name)
                for name, color in colors.items())
names = [name for hsv, name in by_hsv]
len(names)

In [ ]:
tree_name_count = df['수종명(한글명)'].nunique()
tree_name_count

In [ ]:
# 나무 종류별로 다른 색상으로 그리기 위해 컬러 코드 추출하기
start_no = 20
color_names = names[start_no::3]
color_names_list = color_names[:tree_name_count]

In [ ]:
folium_color_dict = dict(zip(df['수종명(한글명)'].unique().tolist(), color_names_list))
folium_color_dict

## folium 을 통한 나무 위치 시각화

In [ ]:
# tooltip이 한줄에 찍히면 보기가 어렵기 때문에 <br/> 태그로 줄바꿈 표시

str(df.loc[0].to_dict()).replace(
    "'수목 수령'", "<br/>'수목 수령'").replace(
    "'법정동 코드'", "<br/>'법정동 코드'").replace(
    "'조성 일자'", "<br/>'조성 일자'")

### 수목 수령이 큰 나무만 보기

In [ ]:
# tree_name = "계수나무"
# tiles="Stamen Toner"
tree_map = folium.Map(location=lat_long.values, zoom_start=12)
# df_tree = df[df['수종명(한글명)'] == tree_name]
df_tree = df[df['수목 수령'] > 100]

for sub_tree in df_tree.iterrows():
    tree = sub_tree[1]
    folium.CircleMarker(location=[tree["위도"], tree["경도"]], 
                        radius=1, color=folium_color_dict[tree['수종명(한글명)']],
                        tooltip=str(tree.to_dict()).replace(
                            "'수목 수령'", "<br/>'수목 수령'").replace(
                            "'법정동 코드'", "<br/>'법정동 코드'").replace(
                            "'조성 일자'", "<br/>'조성 일자'")
                       ).add_to(tree_map)
tree_map.save('tree_age.html')

### 전체 나무

In [ ]:
# tiles="Stamen Toner"
tree_map = folium.Map(location=lat_long.values, zoom_start=12)

for sub_tree in df.iterrows():
    tree = sub_tree[1]
    folium.CircleMarker(location=[tree["위도"], tree["경도"]], 
                        radius=1, color=folium_color_dict[tree['수종명(한글명)']],
                        tooltip=str(tree.to_dict()).replace(
                            "'수목 수령'", "<br/>'수목 수령'").replace(
                            "'법정동 코드'", "<br/>'법정동 코드'").replace(
                            "'조성 일자'", "<br/>'조성 일자'")
                       ).add_to(tree_map)
tree_map.save('tree_all.html')

### 지도의 타일 변경하기

In [ ]:
# tiles="Stamen Toner"
tree_map = folium.Map(location=lat_long.values, zoom_start=12, tiles="Stamen Toner")

for sub_tree in df.iterrows():
    tree = sub_tree[1]
    folium.CircleMarker(location=[tree["위도"], tree["경도"]], 
                        radius=1, color=folium_color_dict[tree['수종명(한글명)']],
                        tooltip=str(tree.to_dict()).replace(
                            "'수목 수령'", "<br/>'수목 수령'").replace(
                            "'법정동 코드'", "<br/>'법정동 코드'").replace(
                            "'조성 일자'", "<br/>'조성 일자'")
                       ).add_to(tree_map)
tree_map.save('stamen_tree_all.html')